Carga de archivo
----

Se lee el archivo con listas de películas y se almacena en mapas. El formato del archivo es:
````
id_lista1, movie1, movie2,...
id_lista2, ...
....
````

In [277]:
import csv 
lsname = "asmr"
filename = "asmr.txt"

movie2list = dict()
list2movie = dict()

mov2index = dict();
all_movies = [] # con repetición
all_movies2 = []; # sin repetición
idm=0


with open(filename) as tsv:
    for mlist in csv.reader(tsv, dialect="excel-tab"):
        while '' in mlist: mlist.remove('')
        id_list = mlist[0]
        list2movie[id_list] = set(mlist[1:])
        for movie in mlist[1:]:
            if movie in movie2list:
                movie2list[movie].add(id_list)
            else: 
                movie2list[movie] = set([id_list])
                
            if movie not in all_movies2:
                mov2index[movie]=idm
                all_movies2.append(movie)
                idm += 1
            all_movies.append(movie)
                
lssize = len(list2movie)
print("Total de peliculas:",len(mov2index))
print("Total de peliculas (contando repetidas):",len(all_movies))
print("Total de listas:",lssize)

Total de peliculas: 1248
Total de peliculas (contando repetidas): 2967
Total de listas: 225


In [19]:
def get_movies(idList):      
    return list2movie[idList]
print(get_movies("asmr1001"))

{'mulholland-drive', 'whiplash-2014', 'uncle-boonmee-who-can-recall-his-past-lives', 'uncut-gems', 'birdman-or-the-unexpected-virtue-of-ignorance', 'where-the-wild-things-are', 'the-diving-bell-and-the-butterfly', 'moonlight-2016', 'roma-2018', 'the-tree-of-life-2011'}


In [233]:
def get_lists(idMovie):      
    return movie2list[idMovie]
print(get_lists("the-tree-of-life-2011"))

{'asmr1141', 'asmr1048', 'asmr1152', 'asmr1175', 'asmr1209', 'asmr1158', 'asmr1078', 'asmr1107', 'asmr1121', 'asmr1053', 'asmr1114', 'asmr1155', 'asmr1086', 'asmr1116', 'asmr1129', 'asmr1146', 'asmr1191', 'asmr1110', 'asmr1197', 'asmr1001', 'asmr1153', 'asmr1050', 'asmr1025', 'asmr1097', 'asmr1172', 'asmr1018', 'asmr1150', 'asmr1060', 'asmr1123', 'asmr1137', 'asmr1026', 'asmr1205', 'asmr1017', 'asmr1177', 'asmr1170'}


mov2vec
---

Esta función transforma una lista de películas `mlist` en un vector `[0 0 0 0 X .... X .. X .. 0]`.
Cada casilla del vector represnta a una película, y las `X=1/len(mlist)` corresponden a las películas de la lista.

In [21]:
import random
import numpy as np
def mov2vec(mlist, mov2index):
    size = len (mov2index)
    vec = np.zeros(size)
    for m in mlist:
        vec[mov2index[m]] = 1.0/len(mlist)
    return vec

Generación de Datos de entrenamiento
----

Cada dato de entrenamiento se genera de la siguiente forma:

1. Se selecciona una lista aleatoria `L` de la primera mitad de listas (`lssize/2`).
2. La entrada (`data_x`) corresponde a una de entre 2 y 5 películas pertenecienta a `L`.
3. La salida (`data_y`) coresponde a la lista de películas completa `L`

Notar que las listas son vectorizadas usando la función `mov2vec`

In [211]:
n_data = 10000
data_x = np.zeros((n_data, len(mov2index))); data_y = np.zeros((n_data, len(mov2index)));
for i in range(0,n_data):
    j = random.randint(1001,1000+int(lssize/2))
    L = list(get_movies(lsname+str(j)))
    sz = len(L)
    cut = random.randint(5,10)
    random.shuffle(L)
    l1 = L[:cut]
    data_x[i] = mov2vec(l1, mov2index)
    data_y[i] = mov2vec(L, mov2index)

Modelo de red neuronal
---

Se crea la red neuronal densa. Tanto la entrada como la salida es un vector de largo igual a la cantidad total de películas.

In [220]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

N = len (mov2index)
def baseline_model():
    # define the keras model
    model = Sequential()
    model.add(Dense(20, input_dim=N, activation='relu'))  # "attributes" layer
    #model.add(Dense(50, activation='relu'))  # "attributes" layer
    model.add(Dense(N))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    return model

Ajuste de la red
---

La red se entrena usando los datos creados más arriba

In [221]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=500, verbose=False)
estimator.fit(data_x, data_y)
#kfold = KFold(n_splits=10)
#results = cross_val_score(estimator, data_x, data_y, cv=kfold)
#print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Recommend
----

La función `recommend` usa la red entrenada para realizar una recomendación.

La función retorna una lista de `n` películas que recomienda a partir de una lista de entrada `F`. Las películas recomendadas no están en el conjunto `F`.

Además se puede incluir una lista `seen` con las películas vistas por el usuario para que no aparezcan entre las recomendaciones.


In [222]:
def recommend(F, n=10, seen=[], verbose=True):
    data_x = mov2vec(F, mov2index)
    
    data_y = estimator.predict([[data_x]])[0]
    
    max_indexes = (-data_y).argsort()
    recommendations = set()
    for i in max_indexes:
        if all_movies2[i] not in F and all_movies2[i] not in seen:
            if(verbose): print(data_y[i], all_movies2[i])
            recommendations.add (all_movies2[i])
            if len(recommendations) == n: break  
    return recommendations

En este ejemplo se muestran las recomendaciones a partir de una lista dada. Adjunto a cada recomendación, el valor asociado a cada película por la red (recuerda que la red retorna un vector en donde cada casilla corresponde a una película, las películas recomendadas son las que maximizan el valor).

> Cabe mencionar que utilicé el recomendador con esta lista F para que me recomendara películas y me han parecido bastante buenas las que he visto hasta ahora.

In [223]:
F = ['little-forest','columbus-2017','kikis-delivery-service','the-assassination-of-jesse-james-by-the-coward-robert-ford']
recommend(F,10,verbose=True); 

0.04554712 portrait-of-a-lady-on-fire
0.040911097 arrival-2016
0.03221978 roma-2018
0.032007195 my-neighbor-totoro
0.03129747 blade-runner-2049
0.026988376 the-tree-of-life-2011
0.02692424 once-upon-a-time-in-hollywood
0.017013665 little-women-2019
0.016400408 in-the-mood-for-love
0.014349551 fantasia


### Ejemplo

En este ejemplo se ve el comportamiento del recomendador usando una de las listas de entrenamiento.

In [224]:
m = list(get_movies(lsname+"1010"))
random.shuffle(m)
l1 = m[:3]
l2 = m[3:]

data = mov2vec(l1, mov2index)

print("Lista original:",m)
print("Lista de entrada:", l1)
rec=recommend(l1,5,verbose=False)
print("Películas recomendadas:",list(rec))
print("Aciertos:",len(rec.intersection(m)),"/5")

Lista original: ['night-is-short-walk-on-girl', 'bait-2019', 'dead-slow-ahead', 'in-the-crosswind', 'boy-the-world', 'upstream-color', 'girls-lost', 'the-fall', 'a-ghost-story-2017', 'melancholia']
Lista de entrada: ['night-is-short-walk-on-girl', 'bait-2019', 'dead-slow-ahead']
Películas recomendadas: ['paterson', 'stalker', 'the-duke-of-burgundy', 'the-lighthouse-2019', '2001-a-space-odyssey']
Aciertos: 0 /5


En este ejemplo se ve el comportamiento del recomendador usando una de las listas **no usadas para entrenar**.

In [225]:
m = list(get_movies(lsname+"1172"))
random.shuffle(m)
l1 = m[:4]
l2 = m[4:]

data = mov2vec(l1, mov2index)
print("Lista original:",m)
print("Lista de entrada:", l1)

rec=recommend(l1,10,verbose=False)
print("Películas recomendadas:",list(rec))
print("Aciertos:",len(rec.intersection(m)),"/10")

Lista original: ['columbus-2017', 'the-witch-2015', 'cold-war-2018', 'clementine-2019', 'heartstone', 'portrait-of-a-lady-on-fire', 'personal-shopper', '2001-a-space-odyssey', 'stalker', 'arrival-2016', 'blade-runner-2049', 'at-eternitys-gate', 'enter-the-void', 'never-rarely-sometimes-always', 'the-florida-project', 'the-lighthouse-2019', 'ocean-waves', 'high-life-2018', 'her', 'la-la-land', 'roma-2018', 'moonlight-2016', 'your-name', 'the-tree-of-life-2011', 'gods-own-country-2017', 'waves-2019', 'a-ghost-story-2017']
Lista de entrada: ['columbus-2017', 'the-witch-2015', 'cold-war-2018', 'clementine-2019']
Películas recomendadas: ['the-lure', 'paterson', 'portrait-of-a-lady-on-fire', 'in-the-mood-for-love', 'my-neighbor-totoro', 'jodhaa-akbar', 'roma-2018', 'the-tree-of-life-2011', 'arrival-2016', '2001-a-space-odyssey']
Aciertos: 5 /10


Validación
----

Para validar que el asunto haga cosas con sentido, se usa la siguiente función de validación.

La función `validate` realiza 10 pruebas aleatorias para cada lista que consisten en lo siguiente:

1. Se seleccionan `nsample` películas de la lista.
2. A partir de estas películas se recomiendan `npred`
3. Se calcula la cantidad de aciertos entre las películas recomendadas y las que no fueron sampleadas de la lista.

Para comparar, también se genera una recomendación aleatoria que se obtiene seleccionando películas aleatorias de una **lista de todas las películas con repetición** (para que así las más repetidas tengan mayor probabilidad).
Luego se calcula la cantidad de aciertos de la misma manera que usando el recomendador "inteligente".




In [275]:
import random
random.seed(1)
def validate(list_name, nsample=3, nrec=5):
    rw_tot_success = 0
    rand_tot_success = 0
    tot_size = 0
    
    L = get_movies(list_name) #se obtienen las pelis de la lista
    for m in L: all_movies.remove(m)
    if verbose: print("movies:", L)

    #se eliminan de M las películas que solo aprecen en esta lista
    L2 = [x for x in L if x in all_movies]
    
    #Se considera el caso sólo si existen al menos nsample+2 películas en M
    if len(L2)>=nsample+2:
        hits = 0
        for i in range(10):
            # recommendation using ANN
            source_movies = random.sample(L2,nsample) #seleccionamos nsample pelis al azar
            if verbose: print("source_movies:",source_movies)

            recommendations = recommend(source_movies, n=nrec, verbose=False) 
            MM = [x for x in L2 if x not in source_movies]
            hits += len(recommendations.intersection(MM))
            rw_tot_success += len(recommendations.intersection(MM))/float(len(L2)-nsample)
            tot_size += len(recommendations)
            if verbose: print("ann_rec:",recommendations)

            # random recommendation
            recommendations = set()
            while len(recommendations) < nrec:
                recommendations=recommendations.union(set(random.sample(all_movies,nrec-len(recommendations))))
            rand_tot_success += len(recommendations.intersection(MM))/float(len(L2)-nsample)
            if verbose: print("rand_rec",recommendations)
        
        print(list_name +"\t"+ "%.1f" %(rand_tot_success*10) +"\t"+ "%.1f" %(rw_tot_success*10) +"\t" \
              + str(hits/10.0) + "/" + str(len(L2)-nsample) +"\t" + str(len(L2)) )

        for m in L: all_movies.append(m)
        return rw_tot_success/10.0,rand_tot_success/10.0
    
    for m in L: all_movies.append(m)
    return None, None

Se ejecuta el experimento de validación con las listas que **no se usaron par el entrenamiento**.

In [278]:
verbose=False
from IPython.display import display, Markdown, Latex
display(Markdown('rand: %acierto de recomendador aleatorio'))
display(Markdown('ann: %acierto de recomendador ANN'))
display(Markdown('hits: promedio de aciertos de las 10 películas recomendadas'))
display(Markdown('lenL: largo de la lista'))

print("lista \t\trand \tann \thits \tlenL")
tot_success = 0.0; tot_rnd_success=0; count=0
for i in range(int(lssize/2)+1,lssize):
    success, rnd_success = validate("asmr"+str(1000+i), nsample=3, nrec=20)
    if success != None:
        tot_success += success
        tot_rnd_success += rnd_success
        count +=1
print("% acierto del recomendador ANN:",  "%.1f" %(tot_success/count*100))
print("% acierto recomendador aleatorio:", "%.1f" %(tot_rnd_success/count*100))

rand: %acierto de recomendador aleatorio

ann: %acierto de recomendador ANN

hits: promedio de aciertos de las 10 películas recomendadas

lenL: largo de la lista

lista 		rand 	ann 	hits 	lenL
asmr1113	10.0	14.3	1.0/7	10
asmr1114	8.0	50.0	2.5/5	8
asmr1115	8.6	24.3	1.7/7	10
asmr1117	13.3	10.0	0.3/3	6
asmr1118	11.7	33.3	2.0/6	9
asmr1119	6.7	28.3	1.7/6	9
asmr1120	3.8	5.8	1.4/24	27
asmr1121	6.0	11.3	1.7/15	18
asmr1122	5.0	15.0	0.3/2	5
asmr1123	5.7	27.1	1.9/7	10
asmr1124	3.6	5.7	1.6/28	31
asmr1125	4.0	32.0	1.6/5	8
asmr1126	7.5	20.0	0.8/4	7
asmr1127	7.4	14.8	4.0/27	30
asmr1128	7.1	12.9	0.9/7	10
asmr1129	26.7	70.0	2.1/3	6
asmr1130	8.6	20.0	1.4/7	10
asmr1131	1.7	6.7	0.4/6	9
asmr1132	6.0	4.0	0.2/5	8
asmr1133	0.0	50.0	1.0/2	5
asmr1134	12.5	12.5	0.5/4	7
asmr1135	4.2	10.0	1.2/12	15
asmr1136	20.0	38.0	1.9/5	8
asmr1137	5.0	18.3	1.1/6	9
asmr1138	3.3	0.0	0.0/6	9
asmr1139	6.7	22.5	2.7/12	15
asmr1140	13.3	10.0	0.3/3	6
asmr1141	5.4	11.5	3.0/26	29
asmr1142	10.0	13.3	0.4/3	6
asmr1143	10.0	50.0	2.5/5	8
asmr1144	11.8	16.4	1.8/11	14
asmr1145	10.0	40.0	0.8/2	5
asmr1146	2.5	10.0	0.4/4	7
asmr1148	16.0	8.0	0.4/5	8
asmr1149	11.7	43.3	2.6/6	9
asmr1150	18.3	31.7	1.9/6	9
asmr1